In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# setup envirn
import os
from pathlib import Path

cache_root = Path("/scratch/jindai/hf_cache")
cache_root.mkdir(parents=True, exist_ok=True)

os.environ["HF_HOME"] = str(cache_root)

os.environ["HF_HUB_CACHE"] = str(cache_root / "hub")
os.environ["TRANSFORMERS_CACHE"] = str(cache_root / "transformers")

print("HF_HOME =", os.environ["HF_HOME"])
print("HF_HUB_CACHE =", os.environ.get("HF_HUB_CACHE"))
print("TRANSFORMERS_CACHE =", os.environ.get("TRANSFORMERS_CACHE"))

HF_HOME = /scratch/jindai/hf_cache
HF_HUB_CACHE = /scratch/jindai/hf_cache/hub
TRANSFORMERS_CACHE = /scratch/jindai/hf_cache/transformers


In [3]:
import os, random, zipfile, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

from transformers import CLIPProcessor, CLIPModel
from safetensors.torch import load_file
from collections import defaultdict
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


CONFIG = {
    "base_seed": 171717,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),

    # trials
    "num_trials": 5,
    "trial_seed_stride": 1000,

    # extraction
    "batch_size": 32,

    # TF-IDF
    "tfidf_max_features": 512,

    # finetuned model folders from Drive
    "clip_finetuned_drive_dir": "clip_finetuned_softmax",
    "sigclip_finetuned_drive_dir": "sigmoidclip_finetuned_bce",
    "llama_sigclip_drive_dir": "llamasigclip_assets",

    "vae_ckpt_relpath": "multimodal_vae_400.pth",
    "llamavae_ckpt_relpath": "llamavae_text_vae.pth",
    
    # Fixed preference profiles
    "num_profiles": 1500,
    "interaction_k": 5,
    "pref_threshold": 0.2,

    # evaluation
    "top_k": 5,
    "kfold_splits": 5,
    "rec_std_mode": "profiles",
    
    "pinsage_num_walks": 50,
    "pinsage_walk_length": 10,
    "pinsage_top_m": 10,
    "pinsage_base_percentile": 90,
}

DEVICE = CONFIG["device"]
print("Using device:", DEVICE)


# REPRODUCIBILITY HELPERS
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_global_seed(CONFIG["base_seed"])

Using device: cpu


In [33]:
import pandas as pd
import numpy as np

TRAIN_DF_PATH = "train_df_fixed_paths.csv"
TEST_DF_PATH  = "test_df_fixed_paths.csv"

FEATURE_ROOT = "features_pgl5"
FEATURE_KEY = "llamavae" # <=sigclip_ft / clip_ft / clip_base / tfidf / resnet50 / llamasigclip / llamavae / vae_mu

TRAIN_IDS_PATH = f"{FEATURE_ROOT}/train_ids.npy"
TEST_IDS_PATH  = f"{FEATURE_ROOT}/test_ids.npy"

XTRAIN_PATH = f"{FEATURE_ROOT}/{FEATURE_KEY}/X_train.npy"
XTEST_PATH  = f"{FEATURE_ROOT}/{FEATURE_KEY}/X_test.npy"

train_df = pd.read_csv(TRAIN_DF_PATH)
test_df  = pd.read_csv(TEST_DF_PATH)

print("train_df:", train_df.shape)
print("test_df :", test_df.shape)

train_ids = np.load(TRAIN_IDS_PATH, allow_pickle=True)
test_ids  = np.load(TEST_IDS_PATH,  allow_pickle=True)

X_train = np.load(XTRAIN_PATH)# (151, 512)
X_test  = np.load(XTEST_PATH)# (38, 512)

print("train_ids:", train_ids.shape, "X_train:", X_train.shape)
print("test_ids :", test_ids.shape,  "X_test :", X_test.shape)

assert "id" in train_df.columns and "id" in test_df.columns
assert len(train_ids) == X_train.shape[0]
assert len(test_ids)  == X_test.shape[0]
assert X_train.shape[1] == X_test.shape[1]

train_df = train_df.set_index("id").loc[train_ids].reset_index()
test_df  = test_df.set_index("id").loc[test_ids].reset_index()

assert (train_df["id"].values == train_ids).all()
assert (test_df["id"].values  == test_ids).all()

print("Loaded + aligned: df rows match feature order")
print("Embedding dim =", X_train.shape[1])

train_df: (151, 10)
test_df : (38, 10)
train_ids: (151,) X_train: (151, 768)
test_ids : (38,) X_test : (38, 768)
Loaded + aligned: df rows match feature order
Embedding dim = 768


In [5]:
X_all = np.vstack([X_train, X_test]).astype(np.float32)
id_all = np.concatenate([train_ids, test_ids])

print("X_all:", X_all.shape, "id_all:", id_all.shape)

X_all: (189, 512) id_all: (189,)


In [6]:
import numpy as np

PREF_PATH = "user_pref_profiles.npz"   

data = np.load(PREF_PATH, allow_pickle=True)
interacted_ids = data["interacted_ids"]
preferred_mat  = data["preferred_mat"]
label_cols     = list(data["label_cols"])

print("Loaded profiles:", interacted_ids.shape, preferred_mat.shape, label_cols)

Loaded profiles: (1500, 5) (1500, 3) ['animal_label', 'myth_label', 'tree_label']


In [7]:
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize

## PyGeometric Implementation from Saurabh code
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd


#GraphSAGE definition
class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels, project=True, normalize=False, aggr="max")
        self.conv2 = SAGEConv(hidden_channels, out_channels, project=True, normalize=False, aggr="max")

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def get_embeddings(self, x, edge_index):
        x = self.conv1(x, edge_index)
        return x

#Build graph from features
def build_graph(features_np):
    sim = cosine_similarity(features_np)
    sim[sim < 0] = 0
    mask = ~np.eye(len(sim), dtype=bool)
    threshold = np.percentile(sim[mask], 90)
    src, dst = np.where((sim >= threshold) & (np.eye(len(sim)) == 0))
    edge_index = torch.tensor([src, dst], dtype=torch.long)
    print(edge_index.shape)
    x = torch.tensor(features_np, dtype=torch.float)
    
    return Data(x=x, edge_index=edge_index)


#Train GNN model
def train_gnn_model(model, data, labels, device, epochs=600):
    model = model.to(device)
    data = data.to(device)
    labels = labels.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.CrossEntropyLoss()

    for _ in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = loss_fn(out, labels)
        loss.backward()
        optimizer.step()

    return model

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

def evaluate_recommendation(
    model, data_train, data_test,
    df_train, df_test,
    interacted_ids, preferred_mat, label_cols,
    label_type,                 # "animal" / "myth" / "tree"
    top_k=5,
    seed=None, 
    n_eval_profiles=1500, 
    profile_sample_mode="subsample"

):
    """
    Profiles-based Precision@K aligned with Transductive:
    - fixed interacted_ids / preferred_mat
    - user query = mean embedding of interacted train items
    - full ranking over all test items
    - relevance for a given label = test_item[label]==1
    - only evaluate profiles whose preferred_mat[:, label]==1
    Returns: (mean, std, n_profiles_used)
    """
    model.eval()

    #get embeddings
    with torch.no_grad():
        emb_train = model.get_embeddings(data_train.x, data_train.edge_index).detach().cpu().numpy()
        emb_test  = model.get_embeddings(data_test.x,  data_test.edge_index).detach().cpu().numpy()

    #map label_type -> column index in preferred_mat
    label_col = f"{label_type}_label"
    label_map = {str(c).lower(): j for j, c in enumerate(label_cols)}
    if label_col.lower() not in label_map:
        raise ValueError(f"label_cols doesn't contain '{label_col}'. label_cols={label_cols}")
    pref_idx = label_map[label_col.lower()]

    #test label vector (0/1)
    if label_col not in df_test.columns:
        raise ValueError(f"df_test missing column '{label_col}'")
    y_test = df_test[label_col].values.astype(int)  # shape (n_test,)

    if "id" not in df_train.columns:
        raise ValueError("df_train must contain 'id' column for profile id mapping.")
    train_id_to_idx = {pid: i for i, pid in enumerate(df_train["id"].values)}
    
    candidate = np.where(preferred_mat[:, pref_idx] == 1)[0]

    if profile_sample_mode == "all" or n_eval_profiles >= len(candidate):
        picked = candidate
    else:
        rng = np.random.default_rng(seed)
        picked = rng.choice(candidate, size=n_eval_profiles, replace=False)

        
    scores = []
    for p in picked:

        ids = interacted_ids[p]
        idxs = [train_id_to_idx[i] for i in ids if i in train_id_to_idx]
        if len(idxs) == 0:
            continue

        user_vec = emb_train[idxs].mean(axis=0, keepdims=True)  # (1,d)

        sims = cosine_similarity(user_vec, emb_test)[0]         # (n_test,)
        top_idx = np.argsort(sims)[-top_k:][::-1]

        relevant = y_test[top_idx].sum()
        scores.append(relevant / top_k)

    if len(scores) == 0:
        return np.nan, np.nan, 0

    scores = np.array(scores, dtype=float)
    return float(scores.mean()), float(scores.std()), int(len(scores))


def evaluate_gnn_acc_binary(model, data_test, y_test_np, device):
    model.eval()
    data_test = data_test.to(device)
    with torch.no_grad():
        logits = model(data_test.x, data_test.edge_index)  # (n_test, 2)
        preds = logits.argmax(dim=1).detach().cpu().numpy()
    return float((preds == y_test_np).mean())


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def vanilla_precision_at_k_profiles(
    E_train, E_test,
    df_train, df_test,
    interacted_ids, preferred_mat, label_cols,
    label_type,
    top_k=5,
    seed=None,
    n_eval_profiles=1500,
    profile_sample_mode="subsample" # "subsample" or "all"
):
    label_col = f"{label_type}_label"
    y_test = df_test[label_col].values.astype(int)

    # label_cols 
    label_map = {str(c).lower(): j for j, c in enumerate(label_cols)}
    pref_idx = label_map[label_col.lower()]
    
    candidate = np.where(preferred_mat[:, pref_idx] == 1)[0]

    if profile_sample_mode == "all" or n_eval_profiles >= len(candidate):
        picked = candidate
    else:
        rng = np.random.default_rng(seed)
        picked = rng.choice(candidate, size=n_eval_profiles, replace=False)

    train_id_to_idx = {pid: i for i, pid in enumerate(df_train["id"].values)}

    scores = []
    for p in picked:
        ids = interacted_ids[p]
        idxs = [train_id_to_idx[i] for i in ids if i in train_id_to_idx]
        if len(idxs) == 0:
            continue

        user_vec = E_train[idxs].mean(axis=0, keepdims=True)
        sims = cosine_similarity(user_vec, E_test)[0]
        top_idx = np.argsort(sims)[-top_k:][::-1]
        scores.append(y_test[top_idx].sum() / top_k)

    if len(scores) == 0:
        return np.nan, np.nan, 0

    scores = np.array(scores, dtype=float)
    return float(scores.mean()), float(scores.std()), int(len(scores))

In [10]:
#for one label type
def evaluate_label_type(
    label_type,                 # "animal" / "myth" / "tree"
    train_df, test_df,
    X_train, X_test,
    device,
    interacted_ids, preferred_mat, label_cols,
    top_k=5,
    epochs=600,
    modality_name="FEATURE",
    seed=None
):
    # labels
    label_col = f"{label_type}_label"
    y_train_np = train_df[label_col].values.astype(int)
    y_test_np  = test_df[label_col].values.astype(int)
    y_train_t  = torch.tensor(y_train_np, dtype=torch.long)

    #Vanilla P@5
    v_mean, v_std, v_used = vanilla_precision_at_k_profiles(
        E_train=X_train,
        E_test=X_test,
        df_train=train_df,
        df_test=test_df,
        interacted_ids=interacted_ids,
        preferred_mat=preferred_mat,
        label_cols=label_cols,
        label_type=label_type,
        top_k=top_k,
        seed=seed,                          
        n_eval_profiles=CONFIG["num_profiles"],     
        profile_sample_mode="subsample",          
    )
    
    #graphs (item-item)
    data_train = build_graph(X_train)
    data_test  = build_graph(X_test)

    #GraphSAGE train (binary classifier)
    model = GraphSAGE(in_channels=X_train.shape[1], hidden_channels=12, out_channels=2)
    model = train_gnn_model(model, data_train, y_train_t, device, epochs=epochs)
    model.eval()

    #GraphSAGE Acc (on fixed test split)
    gnn_acc = evaluate_gnn_acc_binary(model, data_test, y_test_np, device)

    #Logistic Acc
    X_all = np.vstack([X_train, X_test]).astype(np.float32)

    y_all = np.concatenate([
        train_df[label_col].values.astype(int),
        test_df[label_col].values.astype(int)
    ])

    skf = StratifiedKFold(
        n_splits=CONFIG["kfold_splits"],
        shuffle=True,
        random_state=seed, 
    )

    fold_scores = []
    for tr_idx, te_idx in skf.split(X_all, y_all):
        lr = LogisticRegression(max_iter=2000)  
        lr.fit(X_all[tr_idx], y_all[tr_idx])
        fold_scores.append(lr.score(X_all[te_idx], y_all[te_idx]))

    lr_acc = float(np.mean(fold_scores))


    #profiles-based P@5
    p_mean, p_std, n_used = evaluate_recommendation(
        model, data_train, data_test,
        train_df, test_df,
        interacted_ids, preferred_mat, label_cols,
        label_type=label_type,
        top_k=top_k,
        seed=seed,                          
        n_eval_profiles=CONFIG["num_profiles"],
        profile_sample_mode="subsample",
    )

    return {
        "Label": label_type.capitalize(),
        "Modality": modality_name,

        "GraphSAGE_Acc": f"{gnn_acc:.4f}",
        "RawFeat_Acc":   f"{lr_acc:.4f}",

        "RawFeat+GraphSAGE P@5": f"{p_mean:.4f} ± {p_std:.4f}",
        "RawFeat P@5":           f"{v_mean:.4f} ± {v_std:.4f}",

        "n_profiles_used (GNN/Vanilla)": f"{n_used}/{v_used}",
}

In [11]:
all_trial_rows = []

TRIAL_SEEDS = [CONFIG["base_seed"] + i * CONFIG["trial_seed_stride"] for i in range(CONFIG["num_trials"])]

for t, seed in enumerate(TRIAL_SEEDS):
    set_global_seed(seed)  

    rows = []
    for lt in ["animal", "myth", "tree"]:
        rows.append(
            evaluate_label_type(
                lt,
                train_df, test_df,
                X_train, X_test,
                DEVICE,
                interacted_ids, preferred_mat, label_cols,
                top_k=5,
                epochs=600,
                modality_name=FEATURE_KEY,
                seed=seed,
            )
        )

    df_trial = pd.DataFrame(rows)
    df_trial["trial"] = t
    df_trial["seed"] = seed
    all_trial_rows.append(df_trial)

trial_all_df = pd.concat(all_trial_rows, ignore_index=True)
trial_all_df

torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])
torch.Size([2, 2266])
torch.Size([2, 142])


,Label,Modality,GraphSAGE_Acc,RawFeat_Acc,RawFeat+GraphSAGE P@5,RawFeat P@5,n_profiles_used (GNN/Vanilla),trial,seed
0,Animal,sigclip_ft,0.5000,0.6031,0.4445 ± 0.1229,0.6245 ± 0.1789,1340/1340,0,171717
1,Myth,sigclip_ft,0.4474,0.5609,0.5889 ± 0.1608,0.6032 ± 0.1399,1478/1478,0,171717
2,Tree,sigclip_ft,0.5526,0.6560,0.3891 ± 0.1543,0.1453 ± 0.1175,1324/1324,0,171717
3,Animal,sigclip_ft,0.4737,0.6031,0.5288 ± 0.1862,0.6245 ± 0.1789,1340/1340,1,172717
4,Myth,sigclip_ft,0.4737,0.5609,0.6535 ± 0.1693,0.6032 ± 0.1399,1478/1478,1,172717
5,Tree,sigclip_ft,0.5526,0.6560,0.3918 ± 0.2175,0.1453 ± 0.1175,1324/1324,1,172717
6,Animal,sigclip_ft,0.5000,0.6031,0.5113 ± 0.2589,0.6245 ± 0.1789,1340/1340,2,173717
7,Myth,sigclip_ft,0.4474,0.5609,0.6290 ± 0.2076,0.6032 ± 0.1399,1478/1478,2,173717
8,Tree,sigclip_ft,0.5000,0.6560,0.3941 ± 0.2046,0.1453 ± 0.1175,1324/1324,2,173717
9,Animal,sigclip_ft,0.5000,0.6031,0.5387 ± 0.1915,0.6245 ± 0.1789,1340/1340,3,174717


In [12]:
import numpy as np
import pandas as pd
import math

def parse_mean(s):
    if isinstance(s, str) and "±" in s:
        a, b = s.split("±")
        return float(a.strip()), float(b.strip())
    return float(s), 0.0

summary_rows = []
for label in ["Animal", "Myth", "Tree"]:
    sub = trial_all_df[trial_all_df["Label"] == label].copy()

    gnn_acc_vals = sub["GraphSAGE_Acc"].astype(float).values   # or PinSAGE_Acc
    raw_acc_vals = sub["RawFeat_Acc"].astype(float).values

    gnn_acc_mean = float(np.mean(gnn_acc_vals))
    gnn_acc_std  = float(np.std(gnn_acc_vals))

    raw_acc_mean = float(np.mean(raw_acc_vals))
    raw_acc_std  = float(np.std(raw_acc_vals))

    gnn_ms = np.array([parse_mean(x) for x in sub["RawFeat+GraphSAGE P@5"].values], dtype=float)
    raw_ms = np.array([parse_mean(x) for x in sub["RawFeat P@5"].values], dtype=float)

    n_used_str = sub["n_profiles_used (GNN/Vanilla)"].iloc[0]
    n_i = int(str(n_used_str).split("/")[0])

    means_g, stds_g = gnn_ms[:, 0], gnn_ms[:, 1]
    means_r, stds_r = raw_ms[:, 0], raw_ms[:, 1]

    # profiles-mode pooling across trials:
    # mean of (std^2 + mean^2) - overall_mean^2
    gnn_p_mean = float(np.mean(means_g))
    gnn_p_var  = float(np.mean(stds_g**2 + means_g**2) - gnn_p_mean**2)
    gnn_p_std  = float(math.sqrt(max(gnn_p_var, 0.0)))

    raw_p_mean = float(np.mean(means_r))
    raw_p_var  = float(np.mean(stds_r**2 + means_r**2) - raw_p_mean**2)
    raw_p_std  = float(math.sqrt(max(raw_p_var, 0.0)))

    summary_rows.append({
        "Label": label,
        "Modality": sub["Modality"].iloc[0],
        "GraphSAGE_Acc": f"{gnn_acc_mean:.2f} ± {gnn_acc_std:.2f}",
        "RawFeat_Acc":   f"{raw_acc_mean:.2f} ± {raw_acc_std:.2f}",
        "RawFeat+GraphSAGE P@5": f"{gnn_p_mean:.2f} ± {gnn_p_std:.2f}",
        "RawFeat P@5":           f"{raw_p_mean:.2f} ± {raw_p_std:.2f}",
        "n_profiles_used (GNN/Vanilla)": n_used_str,
        "n_trials": len(sub),
    })

summary_df = pd.DataFrame(summary_rows)
summary_df

,Label,Modality,GraphSAGE_Acc,RawFeat_Acc,RawFeat+GraphSAGE P@5,RawFeat P@5,n_profiles_used (GNN/Vanilla),n_trials
0,Animal,sigclip_ft,0.49 ± 0.01,0.60 ± 0.00,0.51 ± 0.20,0.62 ± 0.18,1340/1340,5
1,Myth,sigclip_ft,0.47 ± 0.02,0.56 ± 0.00,0.62 ± 0.18,0.60 ± 0.14,1478/1478,5
2,Tree,sigclip_ft,0.53 ± 0.03,0.66 ± 0.00,0.40 ± 0.21,0.15 ± 0.12,1324/1324,5


In [13]:
import os
import json
from pathlib import Path

OUT_DIR = Path("results_tables")
OUT_DIR.mkdir(parents=True, exist_ok=True)

run_tag = f"itemitem_GraphSAGE_{FEATURE_KEY}_P{5}_E{600}_fixedsplit_fixedprofiles_trials{CONFIG['num_trials']}"

trial_csv = OUT_DIR / f"{run_tag}.trials.csv"
trial_pkl = OUT_DIR / f"{run_tag}.trials.pkl"
summary_csv = OUT_DIR / f"{run_tag}.summary.csv"
summary_pkl = OUT_DIR / f"{run_tag}.summary.pkl"

trial_all_df.to_csv(trial_csv, index=False)
trial_all_df.to_pickle(trial_pkl)

summary_df.to_csv(summary_csv, index=False)
summary_df.to_pickle(summary_pkl)

print("Saved:", trial_csv)
print("Saved:", trial_pkl)
print("Saved:", summary_csv)
print("Saved:", summary_pkl)

meta = {
    "run_tag": run_tag,
    "graph": "item-item",
    "model": "GraphSAGE",
    "feature_key": FEATURE_KEY,
    "modality_name": NAME_MAP.get(FEATURE_KEY, FEATURE_KEY) if "NAME_MAP" in globals() else FEATURE_KEY,
    "top_k": 5,
    "epochs": 600,
    "num_trials": int(CONFIG["num_trials"]),
    "trial_seeds": [int(CONFIG["base_seed"] + i * CONFIG["trial_seed_stride"]) for i in range(CONFIG["num_trials"])],
    "split": "fixed train/test from train_df_fixed_paths.csv + test_df_fixed_paths.csv",
    "profiles": "user_pref_profiles.npz (fixed)",
    "feature_root": str(FEATURE_ROOT) if "FEATURE_ROOT" in globals() else "features_pgl5",
    "saved_files": {
        "trials_csv": str(trial_csv),
        "trials_pkl": str(trial_pkl),
        "summary_csv": str(summary_csv),
        "summary_pkl": str(summary_pkl),
    }
}

meta_path = OUT_DIR / f"{run_tag}.meta.json"
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

print("Saved meta:", meta_path)

Saved: results_tables/itemitem_GraphSAGE_sigclip_ft_P5_E600_fixedsplit_fixedprofiles_trials5.trials.csv
Saved: results_tables/itemitem_GraphSAGE_sigclip_ft_P5_E600_fixedsplit_fixedprofiles_trials5.trials.pkl
Saved: results_tables/itemitem_GraphSAGE_sigclip_ft_P5_E600_fixedsplit_fixedprofiles_trials5.summary.csv
Saved: results_tables/itemitem_GraphSAGE_sigclip_ft_P5_E600_fixedsplit_fixedprofiles_trials5.summary.pkl
Saved meta: results_tables/itemitem_GraphSAGE_sigclip_ft_P5_E600_fixedsplit_fixedprofiles_trials5.meta.json


#PINSAGE BELOW

In [14]:
from collections import defaultdict, Counter

def edge_index_to_adj(edge_index, num_nodes):
    """edge_index: [2, E] torch.LongTensor -> adjacency list dict[int, list[int]]"""
    src = edge_index[0].cpu().numpy()
    dst = edge_index[1].cpu().numpy()
    adj = {i: [] for i in range(num_nodes)}
    for s, d in zip(src, dst):
        if s != d:
            adj[int(s)].append(int(d))
    return adj

def build_rw_edge_index_from_edge_index(edge_index, num_nodes,
                                        num_walks=50, walk_length=10,
                                        top_m=10, seed=171717,
                                        make_undirected=True):
    rng = np.random.default_rng(seed)
    adj = edge_index_to_adj(edge_index, num_nodes)

    rw_edges_src = []
    rw_edges_dst = []

    for start in range(num_nodes):
        visits = Counter()

        neigh = adj.get(start, [])
        if len(neigh) == 0:
            continue

        for _ in range(num_walks):
            cur = start
            for _ in range(walk_length):
                nbs = adj.get(cur, [])
                if len(nbs) == 0:
                    break
                nxt = int(rng.choice(nbs))
                if nxt != start:
                    visits[nxt] += 1
                cur = nxt

        if len(visits) == 0:
            continue

        top_neighbors = [j for j, _ in visits.most_common(top_m)]
        for j in top_neighbors:
            rw_edges_src.append(start)
            rw_edges_dst.append(j)
            if make_undirected:
                rw_edges_src.append(j)
                rw_edges_dst.append(start)

    if len(rw_edges_src) == 0:
        return edge_index

    rw_edge_index = torch.tensor([rw_edges_src, rw_edges_dst], dtype=torch.long)
    return rw_edge_index

def build_rw_graph(features_np,
                   num_walks=50, walk_length=10, top_m=10, seed=171717,
                   base_percentile=90):
    """
    first use cosine-threshold to make base graph，then use RW to build rw graph。
    output Data(x, edge_index_rw)
    """
    sim = cosine_similarity(features_np)
    sim[sim < 0] = 0
    mask = ~np.eye(len(sim), dtype=bool)
    threshold = np.percentile(sim[mask], base_percentile)
    src, dst = np.where((sim >= threshold) & (np.eye(len(sim)) == 0))
    edge_index_base = torch.tensor([src, dst], dtype=torch.long)

    x = torch.tensor(features_np, dtype=torch.float)
    num_nodes = x.size(0)

    edge_index_rw = build_rw_edge_index_from_edge_index(
        edge_index_base, num_nodes,
        num_walks=num_walks, walk_length=walk_length, top_m=top_m,
        seed=seed, make_undirected=True
    )

    return Data(x=x, edge_index=edge_index_rw)

In [15]:
#for one label type
def evaluate_label_type_pinsage(
    label_type,                 # "animal" / "myth" / "tree"
    train_df, test_df,
    X_train, X_test,
    device,
    interacted_ids, preferred_mat, label_cols,
    top_k=5,
    epochs=600,
    modality_name="FEATURE", 
    rw_seed=None,
    seed=None
):
    if seed is None:
        seed = CONFIG["base_seed"]
    if rw_seed is None:
        rw_seed = seed
    # labels
    label_col = f"{label_type}_label"
    y_train_np = train_df[label_col].values.astype(int)
    y_test_np  = test_df[label_col].values.astype(int)
    y_train_t  = torch.tensor(y_train_np, dtype=torch.long)

    #Vanilla P@5
    v_mean, v_std, v_used = vanilla_precision_at_k_profiles(
        E_train=X_train,
        E_test=X_test,
        df_train=train_df,
        df_test=test_df,
        interacted_ids=interacted_ids,
        preferred_mat=preferred_mat,
        label_cols=label_cols,
        label_type=label_type,
        top_k=top_k,
        seed=seed,                          
        n_eval_profiles=CONFIG["num_profiles"],     
        profile_sample_mode="subsample",           
    )
    
    #graphs (item-item)
    data_train = build_rw_graph(
        X_train,
        num_walks=CONFIG["pinsage_num_walks"],
        walk_length=CONFIG["pinsage_walk_length"],
        top_m=CONFIG["pinsage_top_m"],
        seed=rw_seed,
        base_percentile=CONFIG["pinsage_base_percentile"],
    )
    data_test = build_rw_graph(
        X_test,
        num_walks=CONFIG["pinsage_num_walks"],
        walk_length=CONFIG["pinsage_walk_length"],
        top_m=CONFIG["pinsage_top_m"],
        seed=rw_seed + 1,
        base_percentile=CONFIG["pinsage_base_percentile"],
    )


    #Train GNN on RW graph
    model = GraphSAGE(in_channels=X_train.shape[1], hidden_channels=12, out_channels=2)
    model = train_gnn_model(model, data_train, y_train_t, device, epochs=epochs)
    model.eval()

    #GraphSAGE Acc
    gnn_acc = evaluate_gnn_acc_binary(model, data_test, y_test_np, device)

    #Logistic Acc
    X_all = np.vstack([X_train, X_test]).astype(np.float32)
    
    y_all = np.concatenate([
        train_df[label_col].values.astype(int),
        test_df[label_col].values.astype(int)
    ])

    skf = StratifiedKFold(
        n_splits=CONFIG["kfold_splits"],
        shuffle=True,
        random_state=seed,   
    )

    fold_scores = []
    for tr_idx, te_idx in skf.split(X_all, y_all):
        lr = LogisticRegression(max_iter=2000)  
        lr.fit(X_all[tr_idx], y_all[tr_idx])
        fold_scores.append(lr.score(X_all[te_idx], y_all[te_idx]))

    lr_acc = float(np.mean(fold_scores))


    # profiles-based P@5
    p_mean, p_std, n_used = evaluate_recommendation(
        model, data_train, data_test,
        train_df, test_df,
        interacted_ids, preferred_mat, label_cols,
        label_type=label_type,
        top_k=top_k,
        seed=seed,                          
        n_eval_profiles=CONFIG["num_profiles"],
        profile_sample_mode="subsample",
    )

    return {
        "Label": label_type.capitalize(),
        "Modality": modality_name,
        "PinSAGE_Acc": f"{gnn_acc:.4f}",          
        "RawFeat_Acc": f"{lr_acc:.4f}",
        "RawFeat+PinSAGE P@5": f"{p_mean:.4f} ± {p_std:.4f}",
        "RawFeat P@5": f"{v_mean:.4f} ± {v_std:.4f}",
        "n_profiles_used (GNN/Vanilla)": f"{n_used}/{v_used}",
}

In [16]:
all_trial_rows_pinsage = []

TRIAL_SEEDS = [CONFIG["base_seed"] + i * CONFIG["trial_seed_stride"] for i in range(CONFIG["num_trials"])]

for t, seed in enumerate(TRIAL_SEEDS):
    set_global_seed(seed)

    rows = []
    for lt in ["animal", "myth", "tree"]:
        rows.append(
            evaluate_label_type_pinsage(
                lt,
                train_df, test_df,
                X_train, X_test,
                DEVICE,
                interacted_ids, preferred_mat, label_cols,
                top_k=5,
                epochs=600,
                modality_name=FEATURE_KEY,
                rw_seed=seed,
                seed=seed
            )
        )

    df_trial = pd.DataFrame(rows)
    df_trial["trial"] = t
    df_trial["seed"] = seed
    all_trial_rows_pinsage.append(df_trial)

trial_all_df_pinsage = pd.concat(all_trial_rows_pinsage, ignore_index=True)
trial_all_df_pinsage

,Label,Modality,PinSAGE_Acc,RawFeat_Acc,RawFeat+PinSAGE P@5,RawFeat P@5,n_profiles_used (GNN/Vanilla),trial,seed
0,Animal,sigclip_ft,0.5000,0.6031,0.4961 ± 0.1567,0.6245 ± 0.1789,1340/1340,0,171717
1,Myth,sigclip_ft,0.5526,0.5609,0.7244 ± 0.1897,0.6032 ± 0.1399,1478/1478,0,171717
2,Tree,sigclip_ft,0.5526,0.6560,0.4962 ± 0.2261,0.1453 ± 0.1175,1324/1324,0,171717
3,Animal,sigclip_ft,0.5263,0.6031,0.5369 ± 0.2153,0.6245 ± 0.1789,1340/1340,1,172717
4,Myth,sigclip_ft,0.4737,0.5609,0.6899 ± 0.2100,0.6032 ± 0.1399,1478/1478,1,172717
5,Tree,sigclip_ft,0.5526,0.6560,0.3065 ± 0.1839,0.1453 ± 0.1175,1324/1324,1,172717
6,Animal,sigclip_ft,0.5789,0.6031,0.4554 ± 0.2632,0.6245 ± 0.1789,1340/1340,2,173717
7,Myth,sigclip_ft,0.3947,0.5609,0.4943 ± 0.1438,0.6032 ± 0.1399,1478/1478,2,173717
8,Tree,sigclip_ft,0.5263,0.6560,0.5568 ± 0.1669,0.1453 ± 0.1175,1324/1324,2,173717
9,Animal,sigclip_ft,0.5000,0.6031,0.4990 ± 0.1612,0.6245 ± 0.1789,1340/1340,3,174717


In [17]:
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd
import math

def parse_mean(s):
    if isinstance(s, str) and "±" in s:
        a, b = s.split("±")
        return float(a.strip()), float(b.strip())
    return float(s), 0.0

summary_rows = []
for label in ["Animal", "Myth", "Tree"]:
    sub = trial_all_df_pinsage[trial_all_df_pinsage["Label"] == label].copy()

    gnn_acc_vals = sub["PinSAGE_Acc"].astype(float).values   # or PinSAGE_Acc
    raw_acc_vals = sub["RawFeat_Acc"].astype(float).values

    gnn_acc_mean = float(np.mean(gnn_acc_vals))
    gnn_acc_std  = float(np.std(gnn_acc_vals))

    raw_acc_mean = float(np.mean(raw_acc_vals))
    raw_acc_std  = float(np.std(raw_acc_vals))

    gnn_ms = np.array([parse_mean(x) for x in sub["RawFeat+PinSAGE P@5"].values], dtype=float)
    raw_ms = np.array([parse_mean(x) for x in sub["RawFeat P@5"].values], dtype=float)

    n_used_str = sub["n_profiles_used (GNN/Vanilla)"].iloc[0]
    n_i = int(str(n_used_str).split("/")[0])

    means_g, stds_g = gnn_ms[:, 0], gnn_ms[:, 1]
    means_r, stds_r = raw_ms[:, 0], raw_ms[:, 1]

    gnn_p_mean = float(np.mean(means_g))
    gnn_p_var  = float(np.mean(stds_g**2 + means_g**2) - gnn_p_mean**2)
    gnn_p_std  = float(math.sqrt(max(gnn_p_var, 0.0)))

    raw_p_mean = float(np.mean(means_r))
    raw_p_var  = float(np.mean(stds_r**2 + means_r**2) - raw_p_mean**2)
    raw_p_std  = float(math.sqrt(max(raw_p_var, 0.0)))

    summary_rows.append({
        "Label": label,
        "Modality": sub["Modality"].iloc[0],
        "PinSAGE_Acc": f"{gnn_acc_mean:.2f} ± {gnn_acc_std:.2f}",
        "RawFeat_Acc":   f"{raw_acc_mean:.2f} ± {raw_acc_std:.2f}",
        "RawFeat+PinSAGE P@5": f"{gnn_p_mean:.2f} ± {gnn_p_std:.2f}",
        "RawFeat P@5":           f"{raw_p_mean:.2f} ± {raw_p_std:.2f}",
        "n_profiles_used (GNN/Vanilla)": n_used_str,
        "n_trials": len(sub),
    })

summary_df = pd.DataFrame(summary_rows)
summary_df

,Label,Modality,PinSAGE_Acc,RawFeat_Acc,RawFeat+PinSAGE P@5,RawFeat P@5,n_profiles_used (GNN/Vanilla),n_trials
0,Animal,sigclip_ft,0.52 ± 0.03,0.60 ± 0.00,0.49 ± 0.20,0.62 ± 0.18,1340/1340,5
1,Myth,sigclip_ft,0.49 ± 0.08,0.56 ± 0.00,0.64 ± 0.21,0.60 ± 0.14,1478/1478,5
2,Tree,sigclip_ft,0.55 ± 0.01,0.66 ± 0.00,0.46 ± 0.20,0.15 ± 0.12,1324/1324,5


In [18]:
#PinSAGE
import json
from pathlib import Path

OUT_DIR = Path("results_tables")
OUT_DIR.mkdir(parents=True, exist_ok=True)

run_tag = f"itemitem_PinSAGE_{FEATURE_KEY}_P{5}_E{600}_fixedsplit_fixedprofiles_trials{CONFIG['num_trials']}"

trial_csv = OUT_DIR / f"{run_tag}.trials.csv"
trial_pkl = OUT_DIR / f"{run_tag}.trials.pkl"
summary_csv = OUT_DIR / f"{run_tag}.summary.csv"
summary_pkl = OUT_DIR / f"{run_tag}.summary.pkl"

trial_all_df_pinsage.to_csv(trial_csv, index=False)
trial_all_df_pinsage.to_pickle(trial_pkl)

summary_df.to_csv(summary_csv, index=False)
summary_df.to_pickle(summary_pkl)

print("Saved:", trial_csv)
print("Saved:", trial_pkl)
print("Saved:", summary_csv)
print("Saved:", summary_pkl)

meta = {
    "run_tag": run_tag,
    "graph": "item-item",
    "model": "PinSAGE",
    "feature_key": FEATURE_KEY,
    "top_k": 5,
    "epochs": 600,
    "num_trials": int(CONFIG["num_trials"]),
    "trial_seeds": [int(CONFIG["base_seed"] + i * CONFIG["trial_seed_stride"]) for i in range(CONFIG["num_trials"])],
    "split": "fixed train/test from train_df_fixed_paths.csv + test_df_fixed_paths.csv",
    "profiles": "user_pref_profiles.npz (fixed)",
    "saved_files": {
        "trials_csv": str(trial_csv),
        "trials_pkl": str(trial_pkl),
        "summary_csv": str(summary_csv),
        "summary_pkl": str(summary_pkl),
    }
}

meta_path = OUT_DIR / f"{run_tag}.meta.json"
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

print("Saved meta:", meta_path)

Saved: results_tables/itemitem_PinSAGE_sigclip_ft_P5_E600_fixedsplit_fixedprofiles_trials5.trials.csv
Saved: results_tables/itemitem_PinSAGE_sigclip_ft_P5_E600_fixedsplit_fixedprofiles_trials5.trials.pkl
Saved: results_tables/itemitem_PinSAGE_sigclip_ft_P5_E600_fixedsplit_fixedprofiles_trials5.summary.csv
Saved: results_tables/itemitem_PinSAGE_sigclip_ft_P5_E600_fixedsplit_fixedprofiles_trials5.summary.pkl
Saved meta: results_tables/itemitem_PinSAGE_sigclip_ft_P5_E600_fixedsplit_fixedprofiles_trials5.meta.json


GATNEI


In [45]:
#GATNE-I
def _l2_normalize_np(x, eps=1e-12):
    n = np.linalg.norm(x, axis=1, keepdims=True)
    return x / np.clip(n, eps, None)

def build_edges_inductive(train_feats, test_feats, k=30, sim_floor=0.0):
    from sklearn.metrics.pairwise import cosine_similarity

    Xtr = _l2_normalize_np(train_feats.astype(np.float32))
    Xte = _l2_normalize_np(test_feats.astype(np.float32))
    n_tr, n_te = Xtr.shape[0], Xte.shape[0]

    sim_tr = cosine_similarity(Xtr, Xtr)
    np.fill_diagonal(sim_tr, 0.0)
    sim_tr[sim_tr < sim_floor] = 0.0
    kk_tr = min(k, max(n_tr - 1, 1))
    idx_tr = np.argpartition(-sim_tr, kth=kk_tr-1, axis=1)[:, :kk_tr]

    rows_tr = np.repeat(np.arange(n_tr), kk_tr)
    cols_tr = idx_tr.reshape(-1)
    src_tr = np.concatenate([rows_tr, cols_tr], axis=0)
    dst_tr = np.concatenate([cols_tr, rows_tr], axis=0)
    edge_train = torch.from_numpy(np.stack([src_tr, dst_tr], axis=0)).long()

    sim_te = cosine_similarity(Xte, Xtr)  # (n_te, n_tr)
    sim_te[sim_te < sim_floor] = 0.0
    kk_te = min(k, n_tr)
    idx_te = np.argpartition(-sim_te, kth=kk_te-1, axis=1)[:, :kk_te]

    test_nodes = n_tr + np.arange(n_te)
    src_inf = idx_te.reshape(-1)                 # train idx
    dst_inf = np.repeat(test_nodes, kk_te)       # test node id
    edge_train_to_test = torch.from_numpy(np.stack([src_inf, dst_inf], axis=0)).long()

    return Xtr, Xte, edge_train, edge_train_to_test

In [46]:
class NSLoss(nn.Module):
    """negative sampling loss for edges"""
    def __init__(self, num_nodes, num_neg=10):
        super().__init__()
        self.num_nodes = num_nodes
        self.num_neg = num_neg

    def forward(self, src_emb, pos_dst_emb, all_emb):
        B, D = src_emb.shape

        pos_score = (src_emb * pos_dst_emb).sum(dim=1)  
        pos_loss = -F.logsigmoid(pos_score).mean()

        neg_idx = torch.randint(0, all_emb.size(0), (B, self.num_neg), device=all_emb.device)
        neg_emb = all_emb[neg_idx]  
        neg_score = torch.einsum("bd,bkd->bk", src_emb, neg_emb)  
        neg_loss = -F.logsigmoid(-neg_score).mean()

        return pos_loss + neg_loss


class GATNEOneType(nn.Module):

    def __init__(self, num_nodes, emb_dim=128):
        super().__init__()
        self.emb = nn.Embedding(num_nodes, emb_dim)
        nn.init.xavier_uniform_(self.emb.weight)

    def get_emb(self):
        return self.emb.weight

    def forward(self, src_idx, dst_idx):
        src = self.emb(src_idx)
        dst = self.emb(dst_idx)
        return src, dst


def train_gatne_one_type(edge_index_train, num_nodes_train, device,
                         emb_dim=128, lr=1e-3, weight_decay=1e-6,
                         epochs=20, batch_size=8192, num_neg=10,
                         seed=171717):
    g = torch.Generator(device="cpu")
    g.manual_seed(seed)

    model = GATNEOneType(num_nodes_train, emb_dim=emb_dim).to(device)
    nsloss = NSLoss(num_nodes_train, num_neg=num_neg).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    src_all = edge_index_train[0].cpu()
    dst_all = edge_index_train[1].cpu()
    E = src_all.numel()

    for ep in range(epochs):
        perm = torch.randperm(E, generator=g)
        src_shuf = src_all[perm]
        dst_shuf = dst_all[perm]

        model.train()
        losses = []

        for i in range(0, E, batch_size):
            src = src_shuf[i:i+batch_size].to(device, non_blocking=True)
            dst = dst_shuf[i:i+batch_size].to(device, non_blocking=True)

            opt.zero_grad(set_to_none=True)
            src_emb, pos_dst_emb = model(src, dst)
            all_emb = model.get_emb()  # [N,D]
            loss = nsloss(src_emb, pos_dst_emb, all_emb)
            loss.backward()
            opt.step()
            losses.append(loss.item())

        print(f"[GATNE-1type] ep {ep+1}/{epochs} loss={float(np.mean(losses)):.4f}")

    model.eval()
    return model

In [47]:
@torch.no_grad()
def infer_test_by_attach_mean(model, edge_train_to_test, n_train, n_test, device):
    W = model.get_emb().detach()  
    if W.device != device:
        W = W.to(device)

    src = edge_train_to_test[0].to(device)
    dst = edge_train_to_test[1].to(device) - n_train  # map to [0, n_test)

    D = W.size(1)
    out = torch.zeros((n_test, D), device=device)
    cnt = torch.zeros((n_test, 1), device=device)

    out.index_add_(0, dst, W[src])
    cnt.index_add_(0, dst, torch.ones((dst.size(0), 1), device=device))

    out = out / cnt.clamp_min(1.0)
    out = F.normalize(out, dim=1)
    return out.detach().cpu().numpy()

In [48]:
def run_gatne_itemitem_for_feature(feature_key,
                                  k=30, sim_floor=0.0,
                                  emb_dim=128, epochs=20,
                                  lr=1e-3, wd=1e-6,
                                  batch_size=8192, num_neg=10,
                                  seed=171717):
    X_train = np.load(f"features_pgl5/{feature_key}/X_train.npy")
    X_test  = np.load(f"features_pgl5/{feature_key}/X_test.npy")

    Xtr, Xte, edge_train, edge_train_to_test = build_edges_inductive(
        X_train, X_test, k=k, sim_floor=sim_floor
    )

    n_tr, n_te = Xtr.shape[0], Xte.shape[0]

    model = train_gatne_one_type(
        edge_train,
        num_nodes_train=n_tr,
        device=DEVICE,
        emb_dim=emb_dim,
        lr=lr,
        weight_decay=wd,
        epochs=epochs,
        batch_size=batch_size,
        num_neg=num_neg,
        seed=seed
    )

    E_train = model.get_emb().detach().cpu().numpy()
    E_train = E_train / (np.linalg.norm(E_train, axis=1, keepdims=True) + 1e-12)

    E_test = infer_test_by_attach_mean(model, edge_train_to_test, n_tr, n_te, DEVICE)

    return E_train.astype(np.float32), E_test.astype(np.float32)

In [49]:
def eval_gatne_feature(feature_key, label_type, E_train, E_test, seed):
    v_mean, v_std, v_used = vanilla_precision_at_k_profiles(
        E_train=E_train,
        E_test=E_test,
        df_train=train_df,
        df_test=test_df,
        interacted_ids=interacted_ids,
        preferred_mat=preferred_mat,
        label_cols=label_cols,
        label_type=label_type,
        top_k=CONFIG["top_k"],
        seed=seed,
        n_eval_profiles=CONFIG["num_profiles"],
        profile_sample_mode="subsample",
    )
    return v_mean, v_std, v_used

In [50]:
feature_key = "llamavae"  #sigclip_ft / clip_ft / clip_base / tfidf / resnet50 / llamasigclip / llamavae / vae_mu
Etr, Ete = run_gatne_itemitem_for_feature(
    feature_key,
    k=CONFIG.get("graph_knn_k", 30),
    sim_floor=CONFIG.get("graph_sim_floor", 0.0),
    emb_dim=128,
    epochs=20,
    lr=1e-3,
    wd=1e-6,
    batch_size=8192,
    num_neg=10,
    seed=CONFIG["base_seed"]
)

for lt in ["animal", "myth", "tree"]:
    m, s, used = eval_gatne_feature(feature_key, lt, Etr, Ete, seed=CONFIG["base_seed"])
    print(feature_key, lt, f"P@5={m:.4f} ± {s:.4f}", "used=", used)

[GATNE-1type] ep 1/20 loss=1.3918
[GATNE-1type] ep 2/20 loss=1.3895
[GATNE-1type] ep 3/20 loss=1.3893
[GATNE-1type] ep 4/20 loss=1.3861
[GATNE-1type] ep 5/20 loss=1.3838
[GATNE-1type] ep 6/20 loss=1.3842
[GATNE-1type] ep 7/20 loss=1.3802
[GATNE-1type] ep 8/20 loss=1.3791
[GATNE-1type] ep 9/20 loss=1.3785
[GATNE-1type] ep 10/20 loss=1.3749
[GATNE-1type] ep 11/20 loss=1.3719
[GATNE-1type] ep 12/20 loss=1.3698
[GATNE-1type] ep 13/20 loss=1.3660
[GATNE-1type] ep 14/20 loss=1.3640
[GATNE-1type] ep 15/20 loss=1.3598
[GATNE-1type] ep 16/20 loss=1.3558
[GATNE-1type] ep 17/20 loss=1.3530
[GATNE-1type] ep 18/20 loss=1.3477
[GATNE-1type] ep 19/20 loss=1.3446
[GATNE-1type] ep 20/20 loss=1.3375
llamavae animal P@5=0.5591 ± 0.2441 used= 1340
llamavae myth P@5=0.6122 ± 0.1963 used= 1478
llamavae tree P@5=0.3282 ± 0.2352 used= 1324


In [52]:
def parse_mean(s):
    if isinstance(s, str) and "±" in s:
        a, b = s.split("±")
        return float(a.strip()), float(b.strip())
    return float(s), 0.0

FEATURE_KEY = "llamavae"  #：sigclip_ft / clip_ft / clip_base / tfidf / resnet50 / llamasigclip / llamavae / vae_mu

GATNE_DIM = 128
GATNE_EPOCHS = 20
GATNE_NUM_NEG = 10
GATNE_LR = 1e-3
GATNE_WD = 1e-6
GATNE_BATCH = 8192

KNN_K = CONFIG.get("graph_knn_k", 30)
SIM_FLOOR = CONFIG.get("graph_sim_floor", 0.0)

TRIAL_SEEDS = [CONFIG["base_seed"] + i * CONFIG["trial_seed_stride"] for i in range(CONFIG["num_trials"])]

all_trial_rows_gatne = []

for t, seed in enumerate(TRIAL_SEEDS):
    set_global_seed(seed)

    Etr, Ete = run_gatne_itemitem_for_feature(
        FEATURE_KEY,
        k=KNN_K,
        sim_floor=SIM_FLOOR,
        emb_dim=GATNE_DIM,
        epochs=GATNE_EPOCHS,
        lr=GATNE_LR,
        wd=GATNE_WD,
        batch_size=GATNE_BATCH,
        num_neg=GATNE_NUM_NEG,
        seed=seed
    )

    rows = []
    for lt in ["animal", "myth", "tree"]:
        v_mean, v_std, v_used = vanilla_precision_at_k_profiles(
            E_train=Etr,
            E_test=Ete,
            df_train=train_df,
            df_test=test_df,
            interacted_ids=interacted_ids,
            preferred_mat=preferred_mat,
            label_cols=label_cols,
            label_type=lt,
            top_k=CONFIG["top_k"],
            seed=seed,
            n_eval_profiles=CONFIG["num_profiles"],
            profile_sample_mode="subsample",
        )

        X_train = np.load(f"features_pgl5/{FEATURE_KEY}/X_train.npy")
        X_test  = np.load(f"features_pgl5/{FEATURE_KEY}/X_test.npy")

        raw_mean, raw_std, raw_used = vanilla_precision_at_k_profiles(
            E_train=X_train,
            E_test=X_test,
            df_train=train_df,
            df_test=test_df,
            interacted_ids=interacted_ids,
            preferred_mat=preferred_mat,
            label_cols=label_cols,
            label_type=lt,
            top_k=CONFIG["top_k"],
            seed=seed,
            n_eval_profiles=CONFIG["num_profiles"],
            profile_sample_mode="subsample",
        )

        rows.append({
            "Label": lt.capitalize() if lt != "myth" else "Myth",
            "Modality": FEATURE_KEY,
            "GATNE_Acc": np.nan,
            "RawFeat_Acc": np.nan,
            "RawFeat+GATNE P@5": f"{v_mean:.4f} ± {v_std:.4f}",
            "RawFeat P@5": f"{raw_mean:.4f} ± {raw_std:.4f}",
            "n_profiles_used (GATNE/Vanilla)": f"{raw_used}/{raw_used}",
        })


    df_trial = pd.DataFrame(rows)
    df_trial["trial"] = t
    df_trial["seed"] = seed
    all_trial_rows_gatne.append(df_trial)

trial_all_df_gatne = pd.concat(all_trial_rows_gatne, ignore_index=True)
trial_all_df_gatne

[GATNE-1type] ep 1/30 loss=1.3910
[GATNE-1type] ep 2/30 loss=1.3903
[GATNE-1type] ep 3/30 loss=1.3887
[GATNE-1type] ep 4/30 loss=1.3862
[GATNE-1type] ep 5/30 loss=1.3854
[GATNE-1type] ep 6/30 loss=1.3835
[GATNE-1type] ep 7/30 loss=1.3831
[GATNE-1type] ep 8/30 loss=1.3790
[GATNE-1type] ep 9/30 loss=1.3776
[GATNE-1type] ep 10/30 loss=1.3750
[GATNE-1type] ep 11/30 loss=1.3723
[GATNE-1type] ep 12/30 loss=1.3708
[GATNE-1type] ep 13/30 loss=1.3667
[GATNE-1type] ep 14/30 loss=1.3648
[GATNE-1type] ep 15/30 loss=1.3605
[GATNE-1type] ep 16/30 loss=1.3577
[GATNE-1type] ep 17/30 loss=1.3527
[GATNE-1type] ep 18/30 loss=1.3476
[GATNE-1type] ep 19/30 loss=1.3446
[GATNE-1type] ep 20/30 loss=1.3405
[GATNE-1type] ep 21/30 loss=1.3349
[GATNE-1type] ep 22/30 loss=1.3314
[GATNE-1type] ep 23/30 loss=1.3243
[GATNE-1type] ep 24/30 loss=1.3183
[GATNE-1type] ep 25/30 loss=1.3127
[GATNE-1type] ep 26/30 loss=1.3074
[GATNE-1type] ep 27/30 loss=1.2995
[GATNE-1type] ep 28/30 loss=1.2936
[GATNE-1type] ep 29/30 loss=1

,Label,Modality,GATNE_Acc,RawFeat_Acc,RawFeat+GATNE P@5,RawFeat P@5,n_profiles_used (GATNE/Vanilla),trial,seed
0,Animal,llamavae,NaN,NaN,0.5515 ± 0.2507,0.6227 ± 0.2014,1340/1340,0,171717
1,Myth,llamavae,NaN,NaN,0.6107 ± 0.1935,0.6843 ± 0.1049,1478/1478,0,171717
2,Tree,llamavae,NaN,NaN,0.3150 ± 0.2123,0.1273 ± 0.1052,1324/1324,0,171717
3,Animal,llamavae,NaN,NaN,0.5567 ± 0.2540,0.6227 ± 0.2014,1340/1340,1,172717
4,Myth,llamavae,NaN,NaN,0.6124 ± 0.1943,0.6843 ± 0.1049,1478/1478,1,172717
5,Tree,llamavae,NaN,NaN,0.3547 ± 0.2291,0.1273 ± 0.1052,1324/1324,1,172717
6,Animal,llamavae,NaN,NaN,0.5621 ± 0.2518,0.6227 ± 0.2014,1340/1340,2,173717
7,Myth,llamavae,NaN,NaN,0.6008 ± 0.1923,0.6843 ± 0.1049,1478/1478,2,173717
8,Tree,llamavae,NaN,NaN,0.3432 ± 0.2206,0.1273 ± 0.1052,1324/1324,2,173717
9,Animal,llamavae,NaN,NaN,0.5548 ± 0.2549,0.6227 ± 0.2014,1340/1340,3,174717


In [53]:
import math
import pandas as pd
import numpy as np
import json
from pathlib import Path

summary_rows = []
for label in ["Animal", "Myth", "Tree"]:
    sub = trial_all_df_gatne[trial_all_df_gatne["Label"] == label].copy()

    def safe_mean_std(x):
        x = pd.to_numeric(x, errors="coerce").values.astype(float)
        if np.all(np.isnan(x)):
            return (np.nan, np.nan)
        return (float(np.nanmean(x)), float(np.nanstd(x)))

    gatne_acc_mean, gatne_acc_std = safe_mean_std(sub["GATNE_Acc"])
    raw_acc_mean, raw_acc_std     = safe_mean_std(sub["RawFeat_Acc"])

    gatne_ms = np.array([parse_mean(x) for x in sub["RawFeat+GATNE P@5"].values], dtype=float)
    means_g, stds_g = gatne_ms[:, 0], gatne_ms[:, 1]
    gatne_p_mean = float(np.mean(means_g))
    gatne_p_var  = float(np.mean(stds_g**2 + means_g**2) - gatne_p_mean**2)
    gatne_p_std  = float(math.sqrt(max(gatne_p_var, 0.0)))

    raw_p_val = sub["RawFeat P@5"].iloc[0]
    if isinstance(raw_p_val, str) and "±" in raw_p_val:
        raw_ms = np.array([parse_mean(x) for x in sub["RawFeat P@5"].values], dtype=float)
        means_r, stds_r = raw_ms[:, 0], raw_ms[:, 1]
        raw_p_mean = float(np.mean(means_r))
        raw_p_var  = float(np.mean(stds_r**2 + means_r**2) - raw_p_mean**2)
        raw_p_std  = float(math.sqrt(max(raw_p_var, 0.0)))
        raw_p_str  = f"{raw_p_mean:.2f} ± {raw_p_std:.2f}"
    else:
        raw_p_str = np.nan

    n_used_str = sub["n_profiles_used (GATNE/Vanilla)"].iloc[0]

    summary_rows.append({
        "Label": label,
        "Modality": sub["Modality"].iloc[0],
        "GATNE_Acc": f"{gatne_acc_mean:.2f} ± {gatne_acc_std:.2f}" if not np.isnan(gatne_acc_mean) else np.nan,
        "RawFeat_Acc": f"{raw_acc_mean:.2f} ± {raw_acc_std:.2f}" if not np.isnan(raw_acc_mean) else np.nan,
        "RawFeat+GATNE P@5": f"{gatne_p_mean:.2f} ± {gatne_p_std:.2f}",
        "RawFeat P@5": raw_p_str,
        "n_profiles_used (GATNE/Vanilla)": n_used_str,
        "n_trials": len(sub),
    })

summary_df = pd.DataFrame(summary_rows)
summary_df

,Label,Modality,GATNE_Acc,RawFeat_Acc,RawFeat+GATNE P@5,RawFeat P@5,n_profiles_used (GATNE/Vanilla),n_trials
0,Animal,llamavae,NaN,NaN,0.56 ± 0.25,0.62 ± 0.20,1340/1340,5
1,Myth,llamavae,NaN,NaN,0.60 ± 0.19,0.68 ± 0.10,1478/1478,5
2,Tree,llamavae,NaN,NaN,0.34 ± 0.22,0.13 ± 0.11,1324/1324,5


In [54]:
OUT_DIR = Path("results_tables")
OUT_DIR.mkdir(parents=True, exist_ok=True)

run_tag = (
    f"itemitem_GATNE1_{FEATURE_KEY}"
    f"_k{KNN_K}_neg{GATNE_NUM_NEG}_dim{GATNE_DIM}"
    f"_E{GATNE_EPOCHS}_trials{CONFIG['num_trials']}"
)

trial_csv   = OUT_DIR / f"{run_tag}.trials.csv"
trial_pkl   = OUT_DIR / f"{run_tag}.trials.pkl"
summary_csv = OUT_DIR / f"{run_tag}.summary.csv"
summary_pkl = OUT_DIR / f"{run_tag}.summary.pkl"

trial_all_df_gatne.to_csv(trial_csv, index=False)
trial_all_df_gatne.to_pickle(trial_pkl)

summary_df.to_csv(summary_csv, index=False)
summary_df.to_pickle(summary_pkl)

print("Saved:", trial_csv)
print("Saved:", trial_pkl)
print("Saved:", summary_csv)
print("Saved:", summary_pkl)

meta = {
    "run_tag": run_tag,
    "graph": "item-item",
    "model": "GATNE-1type (scheme-A)",
    "feature_key": FEATURE_KEY,
    "top_k": int(CONFIG["top_k"]),
    "gatne": {
        "dim": int(GATNE_DIM),
        "epochs": int(GATNE_EPOCHS),
        "num_neg": int(GATNE_NUM_NEG),
        "lr": float(GATNE_LR),
        "weight_decay": float(GATNE_WD),
        "batch_size": int(GATNE_BATCH),
    },
    "graph_build": {
        "knn_k": int(KNN_K),
        "sim_floor": float(SIM_FLOOR),
        "inductive_attach": "train->test mean neighbor embedding",
    },
    "num_trials": int(CONFIG["num_trials"]),
    "trial_seeds": [int(s) for s in TRIAL_SEEDS],
    "split": "fixed train/test from train_df_fixed_paths.csv + test_df_fixed_paths.csv",
    "profiles": "user_pref_profiles.npz (fixed)",
    "saved_files": {
        "trials_csv": str(trial_csv),
        "trials_pkl": str(trial_pkl),
        "summary_csv": str(summary_csv),
        "summary_pkl": str(summary_pkl),
    }
}

meta_path = OUT_DIR / f"{run_tag}.meta.json"
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

print("Saved meta:", meta_path)

Saved: results_tables/itemitem_GATNE1_llamavae_k30_neg10_dim128_E30_trials5.trials.csv
Saved: results_tables/itemitem_GATNE1_llamavae_k30_neg10_dim128_E30_trials5.trials.pkl
Saved: results_tables/itemitem_GATNE1_llamavae_k30_neg10_dim128_E30_trials5.summary.csv
Saved: results_tables/itemitem_GATNE1_llamavae_k30_neg10_dim128_E30_trials5.summary.pkl
Saved meta: results_tables/itemitem_GATNE1_llamavae_k30_neg10_dim128_E30_trials5.meta.json
